In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the CSV file
data = pd.read_csv("/content/drive/MyDrive/FaceRecognition/fer2013.csv")

In [ ]:
# Extract pixel values and emotions
pixels = data['pixels'].apply(lambda x: np.array(x.split(), dtype=int))
emotions = data['emotion']
usage = data['Usage']  # Assuming there's a column named 'Usage'

In [ ]:
pixels[0].shape

(2304,)

In [ ]:
pixels.shape

(35887,)

In [ ]:
# Reshape the pixel arrays to 2D arrays
pixels_2d = pixels.apply(lambda x: np.reshape(x, (48, 48)))

In [ ]:
# Reshape pixel values to be suitable for input to models
X = np.array(pixels_2d.tolist()) / 255.0

In [ ]:
# Split the data into training and testing sets based on the 'Usage' column
X_train = X[usage == 'Training']
y_train = emotions[usage == 'Training']

X_test = X[usage == 'PrivateTest']
y_test = emotions[usage == 'PrivateTest']

In [ ]:
X_test.shape

(3589, 48, 48)

In [ ]:
X_test[0]

array([[0.66666667, 0.4627451 , 0.39607843, ..., 0.43921569, 0.51372549,
        0.48627451],
       [0.65882353, 0.49019608, 0.42352941, ..., 0.43529412, 0.48627451,
        0.52156863],
       [0.67843137, 0.5254902 , 0.44313725, ..., 0.40392157, 0.44705882,
        0.50196078],
       ...,
       [0.32156863, 0.41568627, 0.38823529, ..., 0.64313725, 0.51764706,
        0.49803922],
       [0.37647059, 0.41176471, 0.38431373, ..., 0.63529412, 0.50980392,
        0.51764706],
       [0.40784314, 0.40392157, 0.38823529, ..., 0.62352941, 0.52156863,
        0.51372549]])

In [ ]:
X_train_tensor=torch.tensor(X_train,dtype=torch.float)
X_test_tensor=torch.tensor(X_test,dtype=torch.float)

y_train_tensor=torch.tensor(y_train.values,dtype=torch.long)
y_test_tensor=torch.tensor(y_test.values,dtype=torch.long)

In [ ]:
class Deep_Emotion(nn.Module):
    def __init__(self):
        '''
        Deep_Emotion class contains the network architecture.
        '''
        super(Deep_Emotion,self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,10,3)
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(10,10,3)
        self.conv4 = nn.Conv2d(10,10,3)
        self.pool4 = nn.MaxPool2d(2,2)

        self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(810,50)
        self.fc2 = nn.Linear(50,7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self,input):
        out = self.stn(input)

        out = F.relu(self.conv1(out))
        out = self.conv2(out)
        out = F.relu(self.pool2(out))

        out = F.relu(self.conv3(out))
        out = self.norm(self.conv4(out))
        out = F.relu(self.pool4(out))

        out = F.dropout(out)
        out = out.view(-1, 810)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [ ]:
# Import necessary libraries
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

# Define batch size, number of epochs, and learning rate
batch_size = 128
num_epochs = 100
learning_rate = 0.005

save_path = "/content/drive/MyDrive/FaceRecognition/model.pth"

# Create TensorDataset and DataLoader for training and test data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Instantiate the model
model = Deep_Emotion()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Using Adam optimizer

# Learning rate scheduler
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)  # Reduce learning rate by a factor of 0.1 every 20 epochs

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_corrects = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.unsqueeze(1))  # Add one dimension for the channel
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels.data)
        total_train += len(labels)

    scheduler.step()  # Update the learning rate scheduler

    torch.save(model.state_dict(), save_path)

    epoch_train_loss = train_loss / total_train
    epoch_train_acc = train_corrects.double() / total_train

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}')


Epoch [1/100], Train Loss: 1.7355, Train Acc: 0.3003
Epoch [2/100], Train Loss: 1.6429, Train Acc: 0.3528
Epoch [3/100], Train Loss: 1.6052, Train Acc: 0.3721
Epoch [4/100], Train Loss: 1.5800, Train Acc: 0.3844
Epoch [5/100], Train Loss: 1.5630, Train Acc: 0.3908
Epoch [6/100], Train Loss: 1.5384, Train Acc: 0.4017
Epoch [7/100], Train Loss: 1.5232, Train Acc: 0.4062
Epoch [8/100], Train Loss: 1.5098, Train Acc: 0.4133
Epoch [9/100], Train Loss: 1.5026, Train Acc: 0.4159
Epoch [10/100], Train Loss: 1.4882, Train Acc: 0.4235
Epoch [11/100], Train Loss: 1.4883, Train Acc: 0.4225
Epoch [12/100], Train Loss: 1.4752, Train Acc: 0.4293
Epoch [13/100], Train Loss: 1.4726, Train Acc: 0.4308
Epoch [14/100], Train Loss: 1.4625, Train Acc: 0.4366
Epoch [15/100], Train Loss: 1.4560, Train Acc: 0.4399
Epoch [16/100], Train Loss: 1.4634, Train Acc: 0.4348
Epoch [17/100], Train Loss: 1.4488, Train Acc: 0.4407
Epoch [18/100], Train Loss: 1.4503, Train Acc: 0.4413
Epoch [19/100], Train Loss: 1.4437, T

In [ ]:
# Instantiate the model
model = Deep_Emotion()

# Load the saved model state dictionary
model_path = '/content/drive/MyDrive/FaceRecognition/model.pth'  # Update with your actual path
model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [ ]:
# Define batch size, number of epochs, and learning rate
batch_size = 128
num_epochs = 100
learning_rate = 0.005

save_path="/content/drive/MyDrive/FaceRecognition/model.pth"

# Create TensorDataset and DataLoader for training and test data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [ ]:

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_corrects = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.unsqueeze(1))  # Add one dimension for the channel
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels.data)
        total_train += len(labels)

    torch.save(model.state_dict(), save_path)

    epoch_train_loss = train_loss / total_train
    epoch_train_acc = train_corrects.double() / total_train

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}')

Epoch [1/100], Train Loss: 1.1856, Train Acc: 0.5508
Epoch [2/100], Train Loss: 1.1919, Train Acc: 0.5472
Epoch [3/100], Train Loss: 1.1821, Train Acc: 0.5552
Epoch [4/100], Train Loss: 1.1820, Train Acc: 0.5559
Epoch [5/100], Train Loss: 1.1839, Train Acc: 0.5559
Epoch [6/100], Train Loss: 1.1815, Train Acc: 0.5532
Epoch [7/100], Train Loss: 1.1816, Train Acc: 0.5556
Epoch [8/100], Train Loss: 1.1814, Train Acc: 0.5535
Epoch [9/100], Train Loss: 1.1763, Train Acc: 0.5570
Epoch [10/100], Train Loss: 1.1795, Train Acc: 0.5563
Epoch [11/100], Train Loss: 1.1802, Train Acc: 0.5511
Epoch [12/100], Train Loss: 1.1751, Train Acc: 0.5564
Epoch [13/100], Train Loss: 1.1743, Train Acc: 0.5570
Epoch [14/100], Train Loss: 1.1837, Train Acc: 0.5545
Epoch [15/100], Train Loss: 1.1772, Train Acc: 0.5554
Epoch [16/100], Train Loss: 1.1771, Train Acc: 0.5548
Epoch [17/100], Train Loss: 1.1754, Train Acc: 0.5563
Epoch [18/100], Train Loss: 1.1726, Train Acc: 0.5588
Epoch [19/100], Train Loss: 1.1763, T

KeyboardInterrupt: 

In [ ]:
save_path="/content/drive/MyDrive/FaceRecognition/model.pth"
torch.save(model.state_dict(), save_path)

In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.unsqueeze(1))  # Add one dimension for the channel
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy}")

Accuracy on test set: 0.48397882418500976
